# Local Visual QA with LLaMA 3.2 Vision
## ABB #2 - Session 3

Code authored by: Shaw Talebi

### imports

In [1]:
import ollama
import gradio as gr
import time

### basic usage

In [2]:
# pull model
ollama.pull('llama3.2-vision')

ProgressResponse(status='success', completed=None, total=None, digest=None)

In [3]:
# interact with model (locally)
stream = ollama.chat(
    model='llama3.2-vision',
    messages=[{
        'role': 'user',
        'content': 'Please describe this image',
        'images': ['content/shaw.jpeg']
    }],
    stream=True,
)

for chunk in stream:
    print(chunk['message']['content'], end='', flush=True)

The image shows a man sitting on a yellow chair in an office break room. The purpose of the image is to show the man's name tag, which reads "Shaw".

* A man sitting on a yellow chair:
	+ The man is wearing a dark gray polo shirt with a white name tag that says "Shaw".
	+ He has his hands clasped together in front of him.
	+ He is looking at the camera and smiling slightly.
* A fireplace with a green chair next to it:
	+ The fireplace is made of wood and has a small fire burning in it.
	+ There is a green chair placed next to the fireplace, which appears to be comfortable and inviting.
* A TV on the wall behind the man:
	+ The TV is mounted on the wall above a wooden cabinet.
	+ It appears to be showing some kind of presentation or video.

Overall, the image suggests that the man is taking a break in an office break room, possibly after attending a meeting or presentation. The presence of a fireplace and comfortable seating suggests that the break room is designed to be a relaxing and 

### gradio UI

In [4]:
# Function to interact with the Ollama model
def stream_chat(message, history):
    """
    Streams the response from the Ollama model and sends it to the Gradio UI.
    
    Args:
        message (str): The user input message.
        history (list): A list of previous conversation messages.
        
    Yields:
        str: The chatbot's response chunk by chunk.
    """
    # Append the user message to the conversation history
    history.append({"role": "user", "content": message["text"], "images":message["files"]})
    
    # Initialize streaming from Ollama
    stream = ollama.chat(
        model='llama3.2-vision',
        messages=history,  # Full chat history including the current user message
        stream=True,
    )
    
    response_text = ""
    for chunk in stream:
        content = chunk['message']['content']
        response_text += content
        yield response_text  # Send the response incrementally to the UI

    # Append the assistant's full response to the history
    history.append({"role": "assistant", "content": response_text})

In [5]:
# Create a Gradio ChatInterface
gr.ChatInterface(
    fn=stream_chat,  # The function handling the chat
    type="messages",  # Using "messages" to enable chat-style conversation
    examples=[{"text": "What is this paper about?", "files": ['content/attention-is-all-you-need.png']}],  # Example inputs
    multimodal=True,
).launch()

* Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.
